In [1]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
# from standard_metrics import run_with_model_intervention, perplexity_under_reconstruction, mean_nonzero_activations
# Create 
# # make an argument parser directly below
# parser = argparse.ArgumentParser()
# parser.add_argument("--model_name", type=str, default="EleutherAI/pythia-70m-deduped")
# parser.add_argument("--layer", type=int, default=4)
# parser.add_argument("--setting", type=str, default="residual")
# parser.add_argument("--l1_alpha", type=float, default=3e-3)
# parser.add_argument("--num_epochs", type=int, default=10)
# parser.add_argument("--model_batch_size", type=int, default=4)
# parser.add_argument("--lr", type=float, default=1e-3)
# parser.add_argument("--kl", type=bool, default=False)
# parser.add_argument("--reconstruction", type=bool, default=False)
# parser.add_argument("--dataset_name", type=str, default="NeelNanda/pile-10k")
# parser.add_argument("--device", type=str, default="cuda:4")

# args = parser.parse_args()
cfg = dotdict()
# cfg.model_name="EleutherAI/pythia-70m-deduped"
# cfg.model_name="usvsnsp/pythia-6.9b-rm-full-hh-rlhf"
cfg.model_name="reciprocate/dahoas-gptj-rm-static"
# cfg.layers=[i for i in range(28)]
layers = 8
cfg.layers=[i for i in range(20,28)]
cfg.setting="residual"
# cfg.tensor_name="gpt_neox.layers.{layer}"
cfg.tensor_name="transformer.h.{layer}"
# linearly interpolate between 8e-4 & 1e-4
# cfg.l1_alpha=torch.linspace(1e-4, 1e-3, 8).tolist()
cfg.l1_alpha=torch.linspace(3e-3, 8e-3, 8).tolist()
cfg.sparsity=None
cfg.model_batch_size=4
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
# cfg.dataset_name="NeelNanda/pile-10k"
# cfg.dataset_name="Skylion007/openwebtext"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
# cfg.device="cpu"

/root/logan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]

In [3]:
# Load in the model
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(cfg.model_name).to(cfg.device)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
model

Loading checkpoint shards: 100%|██████████| 3/3 [00:34<00:00, 11.40s/it]
Some weights of the model checkpoint at reciprocate/dahoas-gptj-rm-static were not used when initializing GPTJForCausalLM: ['score.weight']
- This IS expected if you are initializing GPTJForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTJForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPTJForCausalLM were not initialized from the model checkpoint at reciprocate/dahoas-gptj-rm-static and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [4]:
from datasets import load_dataset, concatenate_datasets
from activation_dataset import chunk_and_tokenize

# Load the datasets
dataset1 = load_dataset("Anthropic/hh-rlhf", split="train")
dataset2 = load_dataset("Elriggs/openwebtext-100k", split="train")

# Rename 'chosen' column in dataset1 to 'text' to match dataset2
dataset1 = dataset1.rename_column("chosen", "text")

# Concatenate the datasets
dataset = concatenate_datasets([dataset1, dataset2])

# Chunk & tokenize the dataset
max_seq_length= 256
dataset, _ = chunk_and_tokenize(dataset, tokenizer, max_length=max_seq_length)
max_tokens = dataset.num_rows*max_seq_length
print(f"Number of tokens: {max_tokens/1e6:.2f}M")

Found cached dataset json (/root/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-a9fdd36e8b50b8fa/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-a9fdd36e8b50b8fa/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-509f39b09b7b4bc3_*_of_00008.arrow


Number of tokens: 149.68M


In [5]:
from torch.utils.data import DataLoader
batch_size = 16
token_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Run 1 datapoint on model to get the activation size (cause don't want to deal w/ different naming schemes in config files)
from baukit import Trace, TraceDict

text = "1"
tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
# Your activation name will be different. In the next cells, we will show you how to find it.
with torch.no_grad():
    with Trace(model, tensor_names[0]) as ret:
        _ = model(tokens)
        representation = ret.output
        # check if instance tuple
        if(isinstance(representation, tuple)):
            representation = representation[0]
        activation_size = representation.shape[-1]
print(f"Activation size: {activation_size}")

Activation size: 4096


In [7]:
from torch import nn
from torchtyping import TensorType


class TiedSAE(nn.Module):
    def __init__(self, activation_size, n_dict_components):
        super().__init__()
        self.encoder = nn.Parameter(torch.empty((n_dict_components, activation_size)))
        nn.init.xavier_uniform_(self.encoder)
        self.encoder_bias = nn.Parameter(torch.zeros((n_dict_components,)))

    def get_learned_dict(self):
        norms = torch.norm(self.encoder, 2, dim=-1)
        return self.encoder / torch.clamp(norms, 1e-8)[:, None]

    def encode(self, batch):
        c = torch.einsum("nd,bd->bn", self.encoder, batch)
        c = c + self.encoder_bias
        c = torch.clamp(c, min=0.0)
        return c

    def decode(self, code: TensorType["_batch_size", "_n_dict_components"]) -> TensorType["_batch_size", "_activation_size"]:
        learned_dict = self.get_learned_dict()
        x_hat = torch.einsum("nd,bn->bd", learned_dict, code)
        return x_hat

    def forward(self, batch: TensorType["_batch_size", "_activation_size"]) -> TensorType["_batch_size", "_activation_size"]:
        c = self.encode(batch)
        x_hat = self.decode(c)
        return x_hat, c

    def n_dict_components(self):
        return self.get_learned_dict().shape[0]

n_dict_components = activation_size*cfg.ratio
all_autoencoders = [TiedSAE(activation_size, n_dict_components) for _ in range(len(tensor_names))]

In [8]:
optimizers = [torch.optim.Adam(autoencoder.parameters(), lr=cfg.lr) for autoencoder in all_autoencoders]

In [9]:
print("WARNING: Only works on tied SAE")
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.model_name}_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")
wandb.init(project="sparse coding", config=dict(cfg), name=wandb_run_name, entity="sparse_coding")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: elriggs (sparse_coding). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb_run_name: reciprocate/dahoas-gptj-rm-static_0108-210734_None


In [10]:
import numpy as np
# Make directory trained_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
model_save_name = cfg.model_name.split("/")[-1]

num_batch = len(token_loader)
log_space = np.logspace(0, np.log10(num_batch), 11)  # 11 to get 10 intervals
save_batches = [int(x) for x in log_space[1:]]  # Skip the first (0th) interval

dead_features = [torch.zeros(n_dict_components) for _ in range(len(tensor_names))]
last_encoders = [autoencoder.encoder.clone().detach() for autoencoder in all_autoencoders]
# max_num_tokens = 100000000
# Freeze model parameters 
model.eval()
model.requires_grad_(False)
for i, batch in enumerate(tqdm(token_loader)):
    tokens = batch["input_ids"].to(cfg.device)
    with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        with TraceDict(model, tensor_names) as ret:
            _ = model(tokens)

    for auto_ind in range(len(tensor_names)):
        # Index into correct autoencoder, optimizer, and tensor_name
        autoencoder = all_autoencoders[auto_ind].to(cfg.device)
        optimizer = optimizers[auto_ind]
        tensor_name = tensor_names[auto_ind]
        dead_feature = dead_features[auto_ind]
        last_encoder = last_encoders[auto_ind]
        l1_alpha = cfg.l1_alpha[auto_ind]

        # Get intermediate layer activations
        representation = ret[tensor_name].output
        if(isinstance(representation, tuple)):
            representation = representation[0]
        layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
        
        # Run through autoencoder
        c = autoencoder.encode(layer_activations)
        x_hat = autoencoder.decode(c)

        # Calculate loss
        reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
        l1_loss = torch.norm(c, 1, dim=-1).mean()
        total_loss = reconstruction_loss + l1_alpha*l1_loss

        # Update dead features
        dead_feature += c.sum(dim=0).cpu()

        
        # Log
        if (i % 50 == 0): # Check here so first check is model w/o change
            # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
            # Above is wrong, should be similarity between encoder and last encoder
        
            num_tokens_so_far = i*max_seq_length*batch_size
            with torch.no_grad():
                # print the norm of layer_activations
                print(f"Layer {auto_ind} | Layer Activation Norm: {torch.norm(layer_activations, 2, dim=-1).mean().cpu().item()}")
                self_similarity = torch.cosine_similarity(autoencoder.encoder, last_encoder.to(cfg.device), dim=-1).mean().cpu().item()
                last_encoders[auto_ind] = autoencoder.encoder.clone().to("cpu")
                sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
                # Count number of dead_features are zero
                num_dead_features = (dead_feature == 0).sum().item()
            print(f"Layer {auto_ind} | Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.2f} | Reconstruction Loss: {reconstruction_loss:.2f} | L1 Loss: {l1_alpha*l1_loss:.2f} | l1_alpha: {l1_alpha:.2e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
            wandb.log({f"Layer {auto_ind}": {
                'Sparsity': sparsity,
                'Dead Features': num_dead_features,
                'Total Loss': total_loss.item(),
                'Reconstruction Loss': reconstruction_loss.item(),
                'L1 Loss': (l1_alpha*l1_loss).item(),
                'l1_alpha': l1_alpha,
                'Tokens': num_tokens_so_far,
                'Self Similarity': self_similarity,
                'step': i}
            })
            dead_feature = torch.zeros(autoencoder.encoder.shape[0])

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Put SAE back on cpu
        all_autoencoders[auto_ind] = autoencoder.to("cpu")

  0%|          | 0/36544 [00:00<?, ?it/s]

Layer 0 | Layer Activation Norm: 326.910888671875
Layer 0 | Sparsity: 8239.1 | Dead Features: 498 | Total Loss: 105.59 | Reconstruction Loss: 40.81 | L1 Loss: 64.78 | l1_alpha: 3.00e-03 | Tokens: 0 | Self Similarity: 1.00
Layer 1 | Layer Activation Norm: 332.73095703125
Layer 1 | Sparsity: 8248.5 | Dead Features: 528 | Total Loss: 123.58 | Reconstruction Loss: 42.21 | L1 Loss: 81.37 | l1_alpha: 3.71e-03 | Tokens: 0 | Self Similarity: 1.00
Layer 2 | Layer Activation Norm: 343.52294921875
Layer 2 | Sparsity: 8123.0 | Dead Features: 573 | Total Loss: 138.88 | Reconstruction Loss: 41.13 | L1 Loss: 97.75 | l1_alpha: 4.43e-03 | Tokens: 0 | Self Similarity: 1.00
Layer 3 | Layer Activation Norm: 352.97528076171875
Layer 3 | Sparsity: 8185.5 | Dead Features: 637 | Total Loss: 160.99 | Reconstruction Loss: 43.19 | L1 Loss: 117.80 | l1_alpha: 5.14e-03 | Tokens: 0 | Self Similarity: 1.00
Layer 4 | Layer Activation Norm: 394.566650390625
Layer 4 | Sparsity: 8156.7 | Dead Features: 764 | Total Loss:

  0%|          | 100/36544 [17:20<106:32:16, 10.52s/it]

Layer 0 | Layer Activation Norm: 326.36920166015625
Layer 0 | Sparsity: 39.7 | Dead Features: 20 | Total Loss: 5.12 | Reconstruction Loss: 3.36 | L1 Loss: 1.76 | l1_alpha: 3.00e-03 | Tokens: 409600 | Self Similarity: 0.91
Layer 1 | Layer Activation Norm: 332.1792907714844
Layer 1 | Sparsity: 21.9 | Dead Features: 26 | Total Loss: 5.49 | Reconstruction Loss: 3.44 | L1 Loss: 2.05 | l1_alpha: 3.71e-03 | Tokens: 409600 | Self Similarity: 0.91
Layer 2 | Layer Activation Norm: 342.9295654296875
Layer 2 | Sparsity: 15.3 | Dead Features: 37 | Total Loss: 6.63 | Reconstruction Loss: 4.45 | L1 Loss: 2.17 | l1_alpha: 4.43e-03 | Tokens: 409600 | Self Similarity: 0.91
Layer 3 | Layer Activation Norm: 352.311279296875
Layer 3 | Sparsity: 18.7 | Dead Features: 58 | Total Loss: 7.18 | Reconstruction Loss: 4.41 | L1 Loss: 2.77 | l1_alpha: 5.14e-03 | Tokens: 409600 | Self Similarity: 0.91
Layer 4 | Layer Activation Norm: 394.54498291015625
Layer 4 | Sparsity: 31.0 | Dead Features: 59 | Total Loss: 7.90 

  1%|          | 200/36544 [35:14<106:13:19, 10.52s/it]

...

  4%|▍         | 1600/36544 [4:42:14<101:49:00, 10.49s/it]

Layer 0 | Layer Activation Norm: 336.3548889160156
Layer 0 | Sparsity: 15.1 | Dead Features: 4 | Total Loss: 3.94 | Reconstruction Loss: 2.71 | L1 Loss: 1.22 | l1_alpha: 3.00e-03 | Tokens: 6553600 | Self Similarity: 1.00
Layer 1 | Layer Activation Norm: 342.10821533203125
Layer 1 | Sparsity: 12.0 | Dead Features: 0 | Total Loss: 4.31 | Reconstruction Loss: 2.84 | L1 Loss: 1.48 | l1_alpha: 3.71e-03 | Tokens: 6553600 | Self Similarity: 1.00
Layer 2 | Layer Activation Norm: 352.849853515625
Layer 2 | Sparsity: 11.1 | Dead Features: 2 | Total Loss: 4.71 | Reconstruction Loss: 2.93 | L1 Loss: 1.77 | l1_alpha: 4.43e-03 | Tokens: 6553600 | Self Similarity: 1.00
Layer 3 | Layer Activation Norm: 362.18304443359375
Layer 3 | Sparsity: 10.9 | Dead Features: 2 | Total Loss: 5.11 | Reconstruction Loss: 3.04 | L1 Loss: 2.07 | l1_alpha: 5.14e-03 | Tokens: 6553600 | Self Similarity: 1.00
Layer 4 | Layer Activation Norm: 405.446533203125
Layer 4 | Sparsity: 10.7 | Dead Features: 3 | Total Loss: 5.87 | 

  5%|▍         | 1700/36544 [4:59:47<102:03:53, 10.55s/it]

Layer 0 | Layer Activation Norm: 328.76983642578125
Layer 0 | Sparsity: 15.7 | Dead Features: 3 | Total Loss: 3.89 | Reconstruction Loss: 2.71 | L1 Loss: 1.18 | l1_alpha: 3.00e-03 | Tokens: 6963200 | Self Similarity: 1.00
Layer 1 | Layer Activation Norm: 334.3890380859375
Layer 1 | Sparsity: 12.3 | Dead Features: 0 | Total Loss: 4.20 | Reconstruction Loss: 2.79 | L1 Loss: 1.42 | l1_alpha: 3.71e-03 | Tokens: 6963200 | Self Similarity: 1.00
Layer 2 | Layer Activation Norm: 345.209716796875
Layer 2 | Sparsity: 11.0 | Dead Features: 2 | Total Loss: 4.62 | Reconstruction Loss: 2.92 | L1 Loss: 1.69 | l1_alpha: 4.43e-03 | Tokens: 6963200 | Self Similarity: 1.00
Layer 3 | Layer Activation Norm: 354.57989501953125
Layer 3 | Sparsity: 10.9 | Dead Features: 2 | Total Loss: 4.98 | Reconstruction Loss: 3.01 | L1 Loss: 1.97 | l1_alpha: 5.14e-03 | Tokens: 6963200 | Self Similarity: 1.00
Layer 4 | Layer Activation Norm: 396.1776123046875
Layer 4 | Sparsity: 10.9 | Dead Features: 3 | Total Loss: 5.69 |

  5%|▍         | 1726/36544 [5:04:22<101:16:38, 10.47s/it]

In [ ]:
wandb.finish()

Layer 0/Dead Features,█▁▁▁▁▁▁▁▁▁▁
Layer 0/L1 Loss,▄▂▅█▁▄▃▄▄▃▃
Layer 0/Reconstruction Loss,▃▄▃█▂▂▃▅▄▁▄
Layer 0/Self Similarity,█▅▄▃▃▃▂▂▂▁▁
Layer 0/Sparsity,▄▂▄█▁▃▃▃▄▁▃
Layer 0/Tokens,▁▂▂▃▄▅▅▆▇▇█
Layer 0/Total Loss,▃▃▃█▁▂▂▄▄▁▃
Layer 0/l1_alpha,▁▁▁▁▁▁▁▁▁▁▁
Layer 1/Dead Features,█▁▁▁▁▁▁▁▁▁▁
Layer 1/L1 Loss,▄▅▄█▃▆▂▄▃▁▂
Layer 1/Reconstruction Loss,▃▃▁█▁▁▂▄▄▁▅


In [ ]:


# Save every autoencoder
for i, autoencoder in enumerate(all_autoencoders):
    save_name = f"{model_save_name}_r{cfg.ratio}_{tensor_names[i]}"  # trim year
    torch.save(autoencoder, f"trained_models/{save_name}_{cfg.layers[i]}.pt")
# # Save model
# torch.save(autoencoder, f"trained_models/{save_name}.pt")